### Campeonato Brasileiro 2023
#### Projeção da classificação final do Campeonato Brasileiro 2023
**Mémória de cálculo da projeção**<br>
*mpm x prm + mpv x prv*<br>

###### **mpm**: média de pontos como mandante;<br>**mpv**: média de pontos como visitante;<br>**prm**: partidas restantes como mandante;<br>**prv**: partidas restantes como visitante.


In [34]:
#importação das biblioteca
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

pd.options.mode.chained_assignment = None

#carregando a tabela
tabela_completa = pd.read_csv(r'C:\Users\DJK8\OneDrive - PETROBRAS\ROBERTO_SCHNEIDER_DJK8\estudos\br23\tabela_br23.csv', sep=';')

#criando uma cópia para remover as partidas não realizadas
tabela = tabela_completa.copy()
tabela = tabela.dropna()

#numero de rodadas jogadas
max_rodada = tabela['rodada'].max()

In [28]:
#construindo historico de classificação
df_consolidado = None
tabela_consolidada = None
projecao_consolidada = None

for i in range(max_rodada):
    rodada = tabela[tabela['rodada']==i+1]

    #definindo resultado da partida
    rodada['vitoria_mandante'] = rodada.apply(lambda x: 1 if(x['gols_mandante'] > x['gols_visitante']) else 0, axis=1)
    rodada['vitoria_visitante'] = rodada.apply(lambda x: 1 if(x['gols_mandante'] < x['gols_visitante']) else 0, axis=1)
    rodada['empate'] = rodada.apply(lambda x: 1 if(x['gols_mandante'] == x['gols_visitante']) else 0, axis=1)

    #resumo
    #jogos como mandante
    df_mandante = rodada[['time_mandante', 'gols_mandante', 'gols_visitante', 'vitoria_mandante', 'empate', 'vitoria_visitante']]
    df_mandante.columns = ['time', 'gols_marcados_mandante', 'gols_sofridos_mandante', 'vitoria_mandante', 'empate_mandante', 'derrota_mandante']
    #total mandante
    df_mandante = df_mandante.groupby(by="time").sum().reset_index()
    #jogos como visitante
    df_visitante = rodada[['time_visitante', 'gols_visitante', 'gols_mandante', 'vitoria_visitante', 'empate', 'vitoria_mandante']]
    df_visitante.columns = ['time', 'gols_marcados_visitante', 'gols_sofridos_visitante', 'vitoria_visitante', 'empate_visitante', 'derrota_visitante']
    #total visitante
    df_visitante = df_visitante.groupby(by="time").sum().reset_index()

    #consolidando os dados
    df_consolidado = pd.concat([df_consolidado, pd.concat([df_mandante, df_visitante], ignore_index=True)], ignore_index=True)
    df_consolidado = df_consolidado.groupby(by="time").sum().reset_index()
    #numeros consolidados
    df_consolidado['jogos_mandante'] = df_consolidado['vitoria_mandante'] + df_consolidado['empate_mandante'] + df_consolidado['derrota_mandante']
    df_consolidado['jogos_visitante'] = df_consolidado['vitoria_visitante'] + df_consolidado['empate_visitante'] + df_consolidado['derrota_visitante']
    df_consolidado['jogos_total'] = df_consolidado['jogos_mandante'] + df_consolidado['jogos_visitante']
    df_consolidado['vitoria_total'] = df_consolidado['vitoria_mandante'] + df_consolidado['vitoria_visitante']
    df_consolidado['empate_total'] = df_consolidado['empate_mandante'] + df_consolidado['empate_visitante']
    df_consolidado['derrota_total'] = df_consolidado['derrota_mandante'] + df_consolidado['derrota_visitante']
    df_consolidado['pontos_mandante'] = df_consolidado['vitoria_mandante'] * 3 + df_consolidado['empate_mandante']
    df_consolidado['media_pontos_mandante'] = df_consolidado['pontos_mandante'] / (df_consolidado['vitoria_mandante'] + df_consolidado['empate_mandante'] + df_consolidado['derrota_mandante'])
    df_consolidado['pontos_visitante'] = df_consolidado['vitoria_visitante'] * 3 + df_consolidado['empate_visitante']
    df_consolidado['media_pontos_visitante'] = df_consolidado['pontos_visitante'] / (df_consolidado['vitoria_visitante'] + df_consolidado['empate_visitante'] + df_consolidado['derrota_visitante'])
    df_consolidado['pontos_total'] = df_consolidado['pontos_mandante'] + df_consolidado['pontos_visitante']
    df_consolidado['media_pontos_total'] = df_consolidado['pontos_total'] / (df_consolidado['vitoria_total'] + df_consolidado['empate_total'] + df_consolidado['derrota_total'])
    df_consolidado['aproveitamento_mandante'] = df_consolidado['pontos_mandante'] / (df_consolidado['jogos_mandante'] * 3)
    df_consolidado['aproveitamento_visitante'] = df_consolidado['pontos_visitante'] / (df_consolidado['jogos_visitante'] * 3)
    df_consolidado['aproveitamento_total'] = df_consolidado['pontos_total'] / (df_consolidado['jogos_total'] * 3)
    df_consolidado['gols_marcados_total'] = df_consolidado['gols_marcados_mandante'] + df_consolidado['gols_marcados_visitante']
    df_consolidado['saldo_gols_mandante'] = df_consolidado['gols_marcados_mandante'] - df_consolidado['gols_sofridos_mandante']
    df_consolidado['saldo_gols_visitante'] = df_consolidado['gols_marcados_visitante'] - df_consolidado['gols_sofridos_visitante']
    df_consolidado['saldo_gols_total'] = df_consolidado['saldo_gols_mandante'] + df_consolidado['saldo_gols_visitante']
    #ordenando a tabela consolidada
    df_consolidado.sort_values(by=['pontos_total', 'vitoria_total', 'saldo_gols_total', 'gols_marcados_total'], ascending=False, inplace=True)
    df_consolidado = df_consolidado.reset_index().reset_index()
    df_consolidado['classificacao'] = df_consolidado['level_0'] + 1
    df_consolidado.drop(columns=['level_0', 'index'], inplace=True)

    df_consolidado = df_consolidado[['classificacao', 'time', 'pontos_total', 'jogos_total', 'vitoria_total', 'empate_total', 'derrota_total', 
                                    'gols_marcados_total', 'saldo_gols_total', 'media_pontos_total', 'aproveitamento_total', 'pontos_mandante',
                                    'jogos_mandante', 'vitoria_mandante', 'empate_mandante', 'derrota_mandante', 'gols_marcados_mandante', 'gols_sofridos_mandante',
                                    'saldo_gols_mandante', 'media_pontos_mandante', 'aproveitamento_mandante', 'pontos_visitante', 'jogos_visitante', 'vitoria_visitante', 
                                    'empate_visitante', 'derrota_visitante', 'gols_marcados_visitante', 'gols_sofridos_visitante', 'saldo_gols_visitante', 
                                    'media_pontos_visitante','aproveitamento_visitante']]
    
    #historico de classificação na tabela consolidada
    #tabela_consolidada = pd.concat([tabela_consolidada, df_consolidado], ignore_index=True)

    #projecao
    df_consolidado['jogos_mandante_restante'] = 19 - df_consolidado['jogos_mandante']
    df_consolidado['jogos_visitante_restante'] = 19 - df_consolidado['jogos_visitante']
    df_consolidado['pontos_projetados'] = round(df_consolidado['jogos_mandante_restante'] * df_consolidado['media_pontos_mandante'] + df_consolidado['jogos_visitante_restante'] * df_consolidado['media_pontos_visitante'], 0)
    df_consolidado['pontos_finais'] = df_consolidado['pontos_projetados'] + df_consolidado['pontos_total']
    df_consolidado.sort_values(by='pontos_finais', ascending=False, inplace=True)
    df_consolidado = df_consolidado.reset_index().reset_index()
    df_consolidado['classificacao'] = df_consolidado['level_0'] + 1
    df_consolidado.drop(columns=['level_0', 'index'], inplace=True)
    df_consolidado['rodada_projecao'] = i + 1

    #historico de classificação na tabela consolidada
    tabela_consolidada = pd.concat([tabela_consolidada, df_consolidado], ignore_index=True)


In [36]:
#grafico
px.line(tabela_consolidada, x='rodada_projecao', y='pontos_finais', color='time', title='Projeção final rodada a rodada')

In [40]:
#projecao na rodada mais recente
ultima_projecao = tabela_consolidada[tabela_consolidada['rodada_projecao'] == max_rodada]
ultima_projecao = ultima_projecao[['classificacao', 'time', 'pontos_finais']]
display(ultima_projecao)

,classificacao,time,pontos_finais
520,1,Botafogo,82.0
521,2,Red Bull Bragantino,69.0
522,3,Flamengo,67.0
523,4,Palmeiras,62.0
524,5,Grêmio,61.0
525,6,Athletico,61.0
526,7,Atlético,61.0
527,8,Fortaleza,58.0
528,9,Fluminense,56.0
529,10,Cuiabá,51.0
